# Остаточные нейронные сети (Residual Networks)

<a name='1'></a>
## 1 - Импорт библиотек

In [ ]:
import tensorflow as tf
import numpy as np
import scipy.misc
from tensorflow.keras.applications.resnet_v2 import ResNet50V2
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet_v2 import preprocess_input, decode_predictions
from tensorflow.keras import layers
from tensorflow.keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.initializers import random_uniform, glorot_uniform, constant, identity
from tensorflow.python.framework.ops import EagerTensor
from matplotlib.pyplot import imshow
import h5py

%matplotlib inline

<a name='2'></a>
## 2 - Деградация обучения глубоких нейронных сетей

При рассмотрении развития архитектур сверточных нейронных сетей можно увидеть, что существовала тенденция к увеличению количества слоев.

* Важным преимуществом очень глубокой сети состоит в том, что она может аппроксимировать сложные функции.
  
* Однако использование более глубокой сети не всегда помогает. Как показала практика, обучению мешает проблема исчезающих градиентов: в очень глубоких сетях значение градиента быстро достигает нуля, что делает градиентный спуск крайне медленным.

* Иногда возможны ситуации, когда вместо затухания градиентов происходит обратная ситуация - их значения становятся крайне большими и случается "взрыв градиентов" - их резкий рост.

* Таким образом, во время обучения может случиться ситуация, когда скорость обучения первых слоев нейронной сети спадает к очень маленьким значениям:

<caption><center><img src="images/vanishing_grad_kiank.png" style="width:450px;height:220px;"></center></caption>
<caption><center><b> Рисунок 1 : <b>Проблема исчезающих градиентов</b>   </center></caption>


<a name='3'></a>
## 3 - Построение ResNet

В ResNets, применяются соединения, позволяющие модели пропускать один или несколько слоев. Их называют "shortcut" или "skip connection":  

<caption><center><img src="images/skip_connection_kiank.png" style="width:650px;height:200px;"></center></caption>
<caption><center><b> Рисунок 2 : Shortcut соединение</b>   </center></caption>

Левое изображение показывает стандартное соединение нейронной сети. На правом изображении введено соединение, позволяющее пропустить выходные значения одного из слоев. Такая структура позволяет некоторым слоям после обучения соответствовать функции тождественного отображения, а значит увеличение слоев не приведет к ухудшению процесса обучения.
 
Для построения сверточной нейронной сети ResNet используется два основных типа блока. Один из них применяется в случае, когда выходные и входные размеры тензоров не изменяются (identity block), а второй (convocational block) - в обратном случае. 

<a name='3-1'></a>
### 3.1 - Identity Block

Рассмотрим стуктуру этого блока:

<caption><center><img src="images/idblock2_kiank.png" style="width:650px;height:150px;"> </center></caption>
<caption><center><b> Рисунок 3 : Identity block</b>   </center></caption>

Как видно из рисунка 3, существует два пути: основной и Shortcut. После каждого сверточного слоя выполняется функция активации. Кроме того, для ускорения обучения применяются слои пакетной нормализации (BatchNorm).

Далее мы реализуем модифицированную версию identity блока, в которой Shortcut соединение пропускает сразу три скрытых слоя. Такая структура показана на рисунке 4.

<caption><center> <img src="images/idblock3_kiank.png" style="width:650px;height:150px;"> </center></caption>
<caption><center><b> Рисунок 4 : Identity block, модифицированная версия </b>   </center></caption>

Рассмотрим составляющие identity блока:

Первая часть основного пути:

- Первый слой CONV2D имеет фильтры $F_1$ с размерностями (1,1) и размером шага (1,1). Padding - "valid". Для инициализации весов используйте seed = 0 и равномерное распределение  `kernel_initializer = Initializer(seed=0)`.
- В первом слое BatchNorm нормализация выполняется по размерности «каналы».
- Затем применяется функцию активации ReLU. 

Вторая часть основного пути:

- Второй слой CONV2D имеет фильтры $F_2$ с размерностями (f,f) и размером шага (1,1). Padding - "same". Для инициализации весов используйте seed = 0 и равномерное распределение  `kernel_initializer = Initializer(seed=0)`.
- Во втором слое BatchNorm нормализация выполняется по размерности «каналы».
- Затем применяется функцию активации ReLU. 

Третья часть основного пути:

- Третий слой CONV2D имеет фильтры $F_3$ с размерностями (1,1) и размером шага (1,1). Padding - "valid". Для инициализации весов используйте seed = 0 и равномерное распределение  `kernel_initializer = Initializer(seed=0)`.
- Во втором слое BatchNorm нормализация выполняется по размерности «каналы».
- Обратите внимание, на данном этапе функция активации не применяется.

Shortcut соединение:

- Далее выполняется сложение `X_shortcut` и выхода третьего слоя `X`.
- Для выполнения сложения тензоров используется следующий синтаксис `Add()([var1,var2])`
- Далее применяется функция активации ReLU. 

Документация:

- Документация для функции [Conv2D](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Conv2D)
- Документация для функции [BatchNormalization](https://www.tensorflow.org/api_docs/python/tf/keras/layers/BatchNormalization) `BatchNormalization(axis = 3)(X, training = training)`. Если параметр training установлен в значение False, его веса не будут изменяться. Это используется, когда модель применяется в режиме выполнения предсказаний, а не обучения.
- Для применения функции активации используйте  `Activation('relu')(X)`
- Документация для функции [Add](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Add)

Кроме того, мы передаем в функцию тип инициализации весов [tensorflow.keras.initializers](https://www.tensorflow.org/api_docs/python/tf/keras/initializers). По умолчанию используется [random_uniform](https://www.tensorflow.org/api_docs/python/tf/keras/initializers/RandomUniform)

In [ ]:
def identity_block(X, f, filters, training=True, initializer=random_uniform):
    """
    Реализация структуры, изображенной на рисунке 4
    
    Аргументы:
    X -- входной тензор с размерностями (m, n_H_prev, n_W_prev, n_C_prev)
    f -- целое число, задает размер фильтра в середине структуры
    filters -- список python, содержащий целые числа. определяет количество фильтров в сверточных слоях блока
    training -- True: Режим обучения
                False: Режим выполнения предсказаний
    initializer -- тип инициализации весов
    
    Выходные значения:
    X -- выходной тензор с размерностями (m, n_H, n_W, n_C)
    """
    
    # Получение количества фильтров
    F1, F2, F3 = ...
    
    # Сохранение входного значения 
    X_shortcut = ...
    
    # Первая часть основного пути
    ## padding = 'valid'
    X = Conv2D(filters=F1, kernel_size=1, strides=(1,1), padding='valid', kernel_initializer=initializer(seed=0))(X)
    X = BatchNormalization(axis = 3)(X, training = training)
    X = ...
    
    # Вторая часть основного пути
    ## padding = 'same'
    X = ...
    X = ...
    X = ...

    # Третья часть основного пути
    ## padding = 'valid'
    X = ...
    X = ...
    
    ## Выполнение сложения и применение функции активации
    X = ...
    X = ...

    return X

In [ ]:
np.random.seed(1)
X1 = np.ones((1, 4, 4, 3)) * -1
X2 = np.ones((1, 4, 4, 3)) * 1
X3 = np.ones((1, 4, 4, 3)) * 3

X = np.concatenate((X1, X2, X3), axis = 0).astype(np.float32)

A3 = identity_block(X, f=2, filters=[4, 4, 3],
                   initializer=lambda seed=0:constant(value=1),
                   training=False)
print('\033[1mWith training=False\033[0m\n')
A3np = A3.numpy()
print(np.around(A3.numpy()[:,(0,-1),:,:].mean(axis = 3), 5))
resume = A3np[:,(0,-1),:,:].mean(axis = 3)
print(resume[1, 1, 0])

print('\n\033[1mWith training=True\033[0m\n')
np.random.seed(1)
A4 = identity_block(X, f=2, filters=[3, 3, 3],
                   initializer=lambda seed=0:constant(value=1),
                   training=True)
print(np.around(A4.numpy()[:,(0,-1),:,:].mean(axis = 3), 5))

**Ожидаемый вывод**

```
With training=False

[[[  0.        0.        0.        0.     ]
  [  0.        0.        0.        0.     ]]

 [[192.71234 192.71234 192.71234  96.85617]
  [ 96.85617  96.85617  96.85617  48.92808]]

 [[578.1371  578.1371  578.1371  290.5685 ]
  [290.5685  290.5685  290.5685  146.78426]]]
96.85617

With training=True

[[[0.      0.      0.      0.     ]
  [0.      0.      0.      0.     ]]

 [[0.40739 0.40739 0.40739 0.40739]
  [0.40739 0.40739 0.40739 0.40739]]

 [[4.99991 4.99991 4.99991 3.25948]
  [3.25948 3.25948 3.25948 2.40739]]]
```

<a name='3-2'></a>
### 3.2 - Convolutional Block

Рассмотрим второй тип блока. Он применяется в случае, когда выходная и выходная размерность тензоров не совпадает. Отличием от identity блока является наличие слоя CONV2D в shortcut соединении. 

<caption><center><img src="images/convblock_kiank.png" style="width:650px;height:150px;"></center></caption>
<caption><center><b> Рисунок 5 : Convolutional block </b>   </center></caption>

* Слой CONV2D в shortcut соединении нужен для изменения размерности тензора с целью выполнения соответствия размерностей с выходным тензором основного пути. 
* За слоем CONV2D в shortcut соединении не применяется функция активации. 
* В данном блоке используется аргумент `initializer` со значением [glorot_uniform](https://www.tensorflow.org/api_docs/python/tf/keras/initializers/GlorotUniform)

Рассмотрим составляющие convolutional блока:

Первая часть основного пути:

- Первый слой CONV2D имеет фильтры $F_1$ с размерностями (1,1) и размером шага (s,s). Padding - "valid". Для инициализации весов используйте seed = 0 `kernel_initializer = Initializer(seed=0)` и распределение `glorot_uniform`.
- В первом слое BatchNorm нормализация выполняется по размерности «каналы».
- Затем применяется функцию активации ReLU. 

Вторая часть основного пути:

- Второй слой CONV2D имеет фильтры $F_2$ с размерностями (f,f) и размером шага (1,1). Padding - "same". Для инициализации весов используйте seed = 0 `kernel_initializer = Initializer(seed=0)` и распределение `glorot_uniform`.
- Во втором слое BatchNorm нормализация выполняется по размерности «каналы».
- Затем применяется функцию активации ReLU. 

Третья часть основного пути:

- Третий слой CONV2D имеет фильтры $F_3$ с размерностями (1,1) и размером шага (s,s). Padding - "valid". Для инициализации весов используйте seed = 0 `kernel_initializer = Initializer(seed=0)` и распределение `glorot_uniform`.
- Во втором слое BatchNorm нормализация выполняется по размерности «каналы».
- Обратите внимание, на данном этапе функция активации не применяется.

Shortcut соединение:
- Применяется слой CONV2D, который имеет фильтры $F_3$ с размерностями (1,1) и размером шага (s,s). Padding - "valid". Для инициализации весов используйте seed = 0 `kernel_initializer = Initializer(seed=0)` и распределение `glorot_uniform`.
- Далее выполняется сложение `X_shortcut` и выхода третьего слоя `X`.
- Для выполнения сложения тензоров используется следующий синтаксис `Add()([var1,var2])`
- Далее применяется функция активации ReLU. 

Документация:

- Документация для функции [Conv2D](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Conv2D)
- Документация для функции [BatchNormalization](https://www.tensorflow.org/api_docs/python/tf/keras/layers/BatchNormalization) `BatchNormalization(axis = 3)(X, training = training)`. Если параметр training установлен в значение False, его веса не будут изменяться. Это используется, когда модель применяется в режиме выполнения предсказаний, а не обучения.
- Для применения функции активации используйте  `Activation('relu')(X)`
- Документация для функции [Add](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Add)

 

In [ ]:
def convolutional_block(X, f, filters, s = 2, training=True, initializer=glorot_uniform):
    """
    Реализация структуры, изображенной на рисунке 5
    
    Аргументы:
    X -- входной тензор с размерностями (m, n_H_prev, n_W_prev, n_C_prev)
    f -- целое число, задает размер фильтра в середине структуры
    filters -- список python, содержащий целые числа. определяет количество фильтров в сверточных слоях блока
    s -- целое число, задает размер шага свертки в блоке
    training -- True: Режим обучения
                False: Режим выполнения предсказаний
    initializer -- тип инициализации весов. По умолчанию используется Glorot uniform инициализация,
                   которую также называют Xavier uniform.
    
    Выходные значения:
    X -- выходной тензор с размерностями (m, n_H, n_W, n_C)
    """
    
    # Получение количества фильтров
    F1, F2, F3 = ...
    
    # Сохранение входного значения 
    X_shortcut = ...
    
    # Первая часть основного пути
    ## glorot_uniform(seed=0)
    X = ...
    X = ...
    X = ...
    
    # Вторая часть основного пути
    X = ...
    X = ...
    X = ...

    # Третья часть основного пути
    X = ...
    X = ...
    
    # Свертка в shortcut соединении
    X_shortcut = ...  
    X_shortcut = ...

    # Выполнение сложения и применение функции активации
    X = ...
    X = ...
    
    return X

In [ ]:
np.random.seed(1)
X1 = np.ones((1, 4, 4, 3)) * -1
X2 = np.ones((1, 4, 4, 3)) * 1
X3 = np.ones((1, 4, 4, 3)) * 3

X = np.concatenate((X1, X2, X3), axis = 0).astype(np.float32)

A = convolutional_block(X, f = 2, filters = [2, 4, 6], training=False)

assert type(A) == EagerTensor, "Use only tensorflow and keras functions"
assert tuple(tf.shape(A).numpy()) == (3, 2, 2, 6), "Wrong shape."
print(A[0])

B = convolutional_block(X, f = 2, filters = [2, 4, 6], training=True)


**Ожидаемый вывод**

```
tf.Tensor(
[[[0.         0.66683817 0.         0.         0.88853896 0.5274254 ]
  [0.         0.65053666 0.         0.         0.89592844 0.49965227]]

 [[0.         0.6312079  0.         0.         0.8636247  0.47643146]
  [0.         0.5688321  0.         0.         0.85534114 0.41709304]]], shape=(2, 2, 6), dtype=float32)
```

<a name='4'></a>  
## 4 - Построение модели ResNet (50 слоев)

Архитектура сети представлена на рисунке 6. Обозначение "ID BLOCK" отвечает за "Identity block," а "x3" означает 3 последовательно идущих друг за другом блока.

<caption><img src="images/resnet_kiank.png" style="width:850px;height:150px;"></center></caption>
<caption><center><b> Рисунок 6 : ResNet-50 </b>   </center></caption>

Архитектура ResNet-50:
- Входной тензор дополняется нулями (3,3) (по три нуля с каждой стороны)
- Часть 1:
    - Слой 2D Convolution с 64 фильтрами размером (7,7), шаг свертки (2,2). 
    - Слой BatchNorm, нормализация по размерности 'channels'.
    - Функция активации ReLU.
    - MaxPooling с размером окна (3,3) и шагом (2,2).
- Часть 2:
    - Convolutional блок, три набора фильтров по [64,64,256] фильтра соответственно, параметр "f" равен 3, параметр "s" равен 1.
    - Далее следует 2 identity блока, в каждом три набора фильтров по [64,64,256] фильтра соответственно, параметр "f" равен 3.
- Часть 3:
    - Convolutional блок, три набора фильтров по [128,128,512] фильтра соответственно, параметр "f" равен 3, параметр "s" равен 2.
    - Далее следует 3 identity блока, в каждом три набора фильтров по [128,128,512] фильтра соответственно, параметр "f" равен 3.
- Часть 4:
    - Convolutional блок, три набора фильтров по [256, 256, 1024] фильтра соответственно, параметр "f" равен 3, параметр "s" равен 2.
    - Далее следует 5 identity блоков, в каждом три набора фильтров по [256, 256, 1024] фильтра соответственно, параметр "f" равен 3.
- Часть 5:
    - Convolutional блок, три набора фильтров по [512, 512, 2048] фильтра соответственно, параметр "f" равен 3, параметр "s" равен 2.
    - Далее следует 2 identity блока, в каждом три набора фильтров по [512, 512, 2048] фильтра соответственно, параметр "f" равен 3.
- Далее применяется слой 2D Average Pooling, размер окна (2,2).
- Далее используется слой 'flatten'.
- Наконец, применяется полносвязый слой (Dense), количество нейронов равно количеству выходных классов.

    
Документация: 
- Average pooling [see reference](https://www.tensorflow.org/api_docs/python/tf/keras/layers/AveragePooling2D)
- Conv2D: [See reference](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Conv2D)
- BatchNorm: [See reference](https://www.tensorflow.org/api_docs/python/tf/keras/layers/BatchNormalization) 
- Zero padding: [See reference](https://www.tensorflow.org/api_docs/python/tf/keras/layers/ZeroPadding2D)
- Max pooling: [See reference](https://www.tensorflow.org/api_docs/python/tf/keras/layers/MaxPool2D)
- Fully connected layer: [See reference](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Dense)
- Addition: [See reference](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Add)

In [ ]:
def ResNet50(input_shape = (64, 64, 3), classes = 6):
    """
    Имплементация архитектуры ResNet50:
    CONV2D -> BATCHNORM -> RELU -> MAXPOOL -> CONVBLOCK -> IDBLOCK*2 -> CONVBLOCK -> IDBLOCK*3
    -> CONVBLOCK -> IDBLOCK*5 -> CONVBLOCK -> IDBLOCK*2 -> AVGPOOL -> FLATTEN -> DENSE 

    Аргументы:
    input_shape -- размер изображений в датасете
    classes -- целое число, количество классов

    Returns:
    model -- класс Model() из Keras
    """
    
    # Устанавливаем входной тензор с размерностью input_shape
    X_input = Input(input_shape)

    
    # Zero-Padding
    X = ...
    
    # Часть 1
    X = ...
    X = ...
    X = ...
    X = ...

    # Часть 2
    X = ...
    X = ...
    X = ...
    
    ## Часть 3
    X = ...
    X = ...
    X = ...
    X = ...
    
    ## Часть 4
    X = ...
    X = ...
    X = ...
    X = ...
    X = ...
    X = ...

    ## Часть 5
    X = ...
    X = ...
    X = ...

    ## AVGPOOL. используйте "X = AveragePooling2D(...)(X)"
    X = ...

    # выходной слой
    X = ...
    X = ...
    
    
    # Создание модели
    model = Model(inputs = X_input, outputs = X)

    return model

In [ ]:
model = ResNet50(input_shape = (64, 64, 3), classes = 6)
print(model.summary())

In [ ]:
model = ResNet50(input_shape = (64, 64, 3), classes = 6)
model.summary()
# Ожидаемый результат представлен в файле result.md

Компиляция модели

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

Теперь все готово для обучения. Используем датасет из Simple_CNN. Самостоятельно допишите код, который загрузит датасет с цифрами и поделит его на части (аналогично Simple_CNN).

In [ ]:
X_train_orig, Y_train_orig, X_test_orig, Y_test_orig = ...

Обучите модель на 10 эпохах с размером батча 32. При обучении на CPU можно сократить до 5-6 эпох.

In [ ]:
model.fit(X_train, Y_train, epochs = 10, batch_size = 32)

Проверка модели на тестовом датасете.

In [ ]:
preds = model.evaluate(X_test, Y_test)
print ("Loss = " + str(preds[0]))
print ("Test Accuracy = " + str(preds[1]))

<a name='5'></a>  
## 5 - Проверка модели на своем изображении

Проверьте работу нейронной сети на своем изображении. Для этого сделайте фотографию и подайте ее на вход нейронной сети.

In [ ]:
img_path = 'photo.jpg'
img = image.load_img(img_path, target_size=(64, 64))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = x/255.0
print('Input image shape:', x.shape)
imshow(img)
prediction = model.predict(x)
print("Class prediction vector [p(0), p(1), p(2), p(3), p(4), p(5)] = ", prediction)
print("Class:", np.argmax(prediction))
